# Telegram Bot 


Создаем телеграм-бота с помощью PyTelegramBotAPI. 

In [85]:
!pip3 install --upgrade pip

In [86]:
!pip3 install pyTelegramBotAPI

Читаем токен из файла.

In [87]:
import telebot

with open('myToken.txt') as file:
    TOKEN = file.read()
myBot = telebot.TeleBot(TOKEN)

Будем работать с сайтом <http://codeforces.com> .

Для доступа к данным нужно послать HTTP-запрос по адресу https://codeforces.com/api/{methodName} с нужным methodName.

Попробуем получить архив со всеми задачами.


In [88]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

CF_api = 'https://codeforces.com/api/'
url_problems = f'{CF_api}problemset.problems'
response = requests.get(url_problems, params = {'lang' : 'ru'})

Проверим, что запрос успешен.

In [89]:
response

<Response [200]>

In [90]:
response.json()['result'].keys()

dict_keys(['problems', 'problemStatistics'])

По нашему запросу возвращается список объектов Problem и список объектов ProblemStatistics. 

Посмотрим в каком виде к нам приходит задача.

In [91]:
tasks = response.json()['result']['problems']
tasks[0]

{'contestId': 1550,
 'index': 'F',
 'name': 'Попрыгунчик',
 'rating': 2700,
 'tags': ['binary search',
  'data structures',
  'divide and conquer',
  'dp',
  'dsu',
  'graphs',
  'shortest paths'],
 'type': 'PROGRAMMING'}

Структурируем данные со всех задач для будующего датафрейма.

Для каждой задачи хотелось бы знать ссылку для будующего парсинга ее html странички, а также рейтинг и список тегов (темы, под которые задача подходит).

In [93]:
from tqdm.notebook import tqdm

names = []
urls = []
tags = []
ratings = []

for task in tqdm(tasks):
    name = task['name']
    task_id = task['contestId']
    task_index = task['index']
    url = f'https://codeforces.com/problemset/problem/{task_id}/{task_index}'
    tag = task['tags']
    if 'rating' in task.keys():
        rating = task['rating']
        ratings.append(rating)
    else:
        ratings.append('')

    names.append(name)
    urls.append(url)
    tags.append(tag)

Теперь сформируем датайфрейм из задачек.

In [94]:
myTasks = pd.DataFrame(
    {
        'name': names,
        'url': urls,
        'tags': tags,
        'rating' : ratings,
    }
)
myTasks.drop_duplicates(subset=['url'], inplace=True)
myTasks

,name,url,tags,rating
0,Попрыгунчик,https://codeforces.com/problemset/problem/1550/F,"[binary search, data structures, divide and co...",2700
1,Stringforces,https://codeforces.com/problemset/problem/1550/E,"[binary search, bitmasks, dp, greedy, strings,...",2500
2,Отличные массивы,https://codeforces.com/problemset/problem/1550/D,"[combinatorics, constructive algorithms, imple...",2300
3,Манхэттенские подмассивы,https://codeforces.com/problemset/problem/1550/C,"[brute force, geometry, greedy, implementation]",1700
4,Удаление максимальной стоимости,https://codeforces.com/problemset/problem/1550/B,"[greedy, math]",1000
...,...,...,...,...
7061,Наименее круглый путь,https://codeforces.com/problemset/problem/2/B,"[dp, math]",2000
7062,Победитель,https://codeforces.com/problemset/problem/2/A,"[hashing, implementation]",1500
7063,Древнеберляндский цирк,https://codeforces.com/problemset/problem/1/C,"[geometry, math]",2100
7064,Электронные таблицы,https://codeforces.com/problemset/problem/1/B,"[implementation, math]",1600


In [95]:
myTasks.to_csv('myTasks.csv', encoding='utf-8', index=False)
myTasks = pd.read_csv('myTasks.csv')
myTasks

,name,url,tags,rating
0,Попрыгунчик,https://codeforces.com/problemset/problem/1550/F,"['binary search', 'data structures', 'divide a...",2700.0
1,Stringforces,https://codeforces.com/problemset/problem/1550/E,"['binary search', 'bitmasks', 'dp', 'greedy', ...",2500.0
2,Отличные массивы,https://codeforces.com/problemset/problem/1550/D,"['combinatorics', 'constructive algorithms', '...",2300.0
3,Манхэттенские подмассивы,https://codeforces.com/problemset/problem/1550/C,"['brute force', 'geometry', 'greedy', 'impleme...",1700.0
4,Удаление максимальной стоимости,https://codeforces.com/problemset/problem/1550/B,"['greedy', 'math']",1000.0
...,...,...,...,...
7061,Наименее круглый путь,https://codeforces.com/problemset/problem/2/B,"['dp', 'math']",2000.0
7062,Победитель,https://codeforces.com/problemset/problem/2/A,"['hashing', 'implementation']",1500.0
7063,Древнеберляндский цирк,https://codeforces.com/problemset/problem/1/C,"['geometry', 'math']",2100.0
7064,Электронные таблицы,https://codeforces.com/problemset/problem/1/B,"['implementation', 'math']",1600.0


Попробуем сгенерировать задачку с какой-то заданной темой.

Например с тегом 'brute force'.

In [96]:
search_index = myTasks.apply(
    lambda row: 'brute force' in row['tags'],
    axis=1,
)
myTasks[search_index].sample(1)

,name,url,tags,rating
1763,Кёрк и бинарная строка (лёгкая версия),https://codeforces.com/problemset/problem/1204/D1,"['brute force', 'greedy', 'strings']",2000.0


Круто, все получилось!

Теперь попробуем сгенерировать задачу с заданным уровнем сложности.

In [97]:
min_ = 1000
max_ = 2000
search_index = myTasks.apply(
    lambda row: min_ < row['rating'] < max_,
    axis=1,
)
myTasks[search_index].sample(1)

,name,url,tags,rating
5611,Кубики,https://codeforces.com/problemset/problem/328/B,['greedy'],1500.0


Соберем полученную информацию в одну функцию, которая будет выдавать пулл задач.

In [99]:
def make_myTasks(tasks):
    names = []
    urls = []
    tags = []
    ratings = []

    for task in tqdm(tasks):
        name = task['name']
        task_id = task['contestId']
        task_index = task['index']
        url = f'https://codeforces.com/problemset/problem/{task_id}/{task_index}'
        tag = task['tags']
        if 'rating' in task.keys():
            rating = task['rating']
            ratings.append(rating)
        else:
            ratings.append('')

        names.append(name)
        urls.append(url)
        tags.append(tag)

    myTasks = pd.DataFrame(
        {
            'name': names,
            'url': urls,
            'tags': tags,
            'rating' : ratings,
        }
    )
    myTasks.drop_duplicates(subset=['url'], inplace=True)
    
    myTasks.to_csv('myTasks.csv', encoding='utf-8', index=False)
    myTasks = pd.read_csv('myTasks.csv')
    return myTasks

Также сделаем функцию выбора рейтинга, немного модифицировав код, чтобы в случае отсутствия задачи с нужными параметрами, функция не падала.

In [100]:
import numpy

def choose_rating(myTasks, min_, max_):
    search_index = myTasks.apply(
        lambda row: min_ < row['rating'] < max_,
        axis=1,
    )
    if myTasks[search_index].shape[0] == 0:
        return numpy.eye(17)
    return myTasks[search_index].sample(1)

Создадим класс, объекты которого могут генерировать задачи с заданными тегами и сложностью.

In [101]:
class SampleTask:
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        
    @staticmethod
    def from_csv(csv_path: str):
        return SampleTask(pd.read_csv(csv_path))


    def __call__(self, tags='', min_rating=None, max_rating=None):
        CF_api = 'https://codeforces.com/api/'
        url_problems = f'{CF_api}problemset.problems'
        if (len(tags) > 0):
          url_problems = f'{CF_api}problemset.problems?tags={tags}'
        response = requests.get(url_problems, params = {'lang' : 'ru'})

        tasks = response.json()['result']['problems']
        myTasks = make_myTasks(tasks)
        if myTasks.shape[0] == 0:
            return numpy.eye(17)
        if min_rating is not None and max_rating is not None:
            return choose_rating(myTasks, min_rating, max_rating)
            
        return myTasks.sample(1)


In [102]:
request = SampleTask.from_csv('myTasks.csv')

Попробуем сгенерировать задачку.

In [103]:
task_ = request('dp', 0, 2000)
task_

,name,url,tags,rating
1213,Егор и граф,https://codeforces.com/problemset/problem/295/B,"['dp', 'graphs', 'shortest paths']",1700.0


Проверим, что будет если искать несуществующую задачу.

In [104]:
task_ = request('fft', 0, 200)
task_

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0

В этом случае нам вернулась не задача, а единичная матрица размера $17 \times 17$.

Данное решение было принято для того, чтобы можно было по результату работы понять, нашли мы нужную задачу или нет. Проверять это можно будет с помощью 
\begin{equation*}
task\_.shape[1]  = 
 \begin{cases}
   1, \text{если задача существует}\\
   17, \text{иначе}
 \end{cases}
\end{equation*}

Теперь попробуем парсить страничку с задачей, чтобы достать оттуда условие.

Будем делать это с помощью BeautifulSoup.

In [105]:
task_ = request('dp', 0, 2000)
# берем ссылку
url = task_.iloc[0]['url']
response = requests.get(url, params = {'lang' : 'ru'})
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN">
<html lang="ru">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="5b7571f0f341d9dda4781f11fd619995" name="X-Csrf-Token">
   <meta content="width=device-width, initial-scale=0.01" id="viewport" name="viewport"/>
   <script src="//codeforces.org/s/66284/js/jquery-1.8.3.js" type="text/javascript">
   </script>
   <script type="application/javascript">
    window.standaloneContest = false;
        function adjustViewport() {
            var screenWidthPx = Math.min($(window).width(), window.screen.width);
            var siteWidthPx = 1100; // min width of site
            var ratio = Math.min(screenWidthPx / siteWidthPx, 1.0);
            var viewport = "width=device-width, initial-scale=" + ratio;
            $('#viewport').attr('content', viewport);
            var style = $('<style>html * { max-height: 1000000px; }</style>');
            $('html > head').append(style);
        }

        

Вытащим из html кода нужную информацию : название задачи, лимит по времени и памяти и, конечно, условие.

In [106]:
# название
soup.find_all('div', 'problem-statement')[0].find_all('div', 'title')[0].text

'C. Переставь цифры'

In [107]:
def get_limit_time(soup):
    text_time_limit  = ''
    for text in soup.find_all('div', 'time-limit'):
        text_time_limit += text.text
        text_time_limit += '\n'
    index_time = 0
    for i in range(len(text_time_limit)):
        if text_time_limit[i].isdigit():
            index_time = i
            break
    text_time_limit = text_time_limit[ : index_time] + ': ' + text_time_limit[index_time : ]
    return text_time_limit

get_limit_time(soup)

'ограничение по времени на тест: 1 секунда\n'

In [108]:
def get_limit_memory(soup):
    text_memory_limit  = ''
    for text in soup.find_all('div', 'memory-limit'):
        text_memory_limit += text.text
        text_memory_limit += '\n'
    index_memory = 0
    for i in range(len(text_memory_limit)):
        if text_memory_limit[i].isdigit():
            index_memory = i
            break
    text_memory_limit = text_memory_limit[ : index_memory] + ': ' + text_memory_limit[index_memory : ]
    return text_memory_limit

get_limit_memory(soup)

'ограничение по памяти на тест: 256 мегабайт\n'

Напишем функцию, которая будет возвращать нам условие задачи.

In [109]:
myReplace = {'$': '', '\le': ' ≤ ', '\ge': ' ≥ ', '\oplus': '⊕', '\otimes': '⊗', '\times': '×', '\cdot': '·',
             '\pm': '±', '\cap': '∩', '\cup': '∪', '\sum': 'Σ', '\prod': 'П', '\mathrm' : '', '\neq' : '≠',
             '\ne' : '≠', '\ldots' : '...', '\quad' : '  ', '\sqrt' : '√', '\dots' : '...', '\in' : '∈',
             '\notin' : '∉', '\{' : '{', '\}' : '}', '\,' : ''}


def GetTask(task):
    url = task.iloc[0]['url']
    response = requests.get(url, params = {'lang' : 'ru'})
    soup = BeautifulSoup(response.content, 'html.parser')

    text = ''
    text = text + soup.find_all('div', 'problem-statement')[0].find_all('div', 'title')[0].text + '\n\n'
    text = text + get_limit_time(soup) + '\n\n' + get_limit_memory(soup) + '\n\n'

    for line in soup.find_all('p'):
        line_text = line.text
        for key in myReplace: 
            line_text = line_text.replace(key, myReplace[key])
        text += line_text
        text += '\n'

    return text

print(GetTask(task_))

C. Переставь цифры

ограничение по времени на тест: 1 секунда


ограничение по памяти на тест: 256 мегабайт


Даны два натуральных числа a и b. Вам необходимо переставить цифры числа a таким образом, чтобы получилось наибольшее возможное число, не превосходящее b. Никакое из входных или выходных чисел не может начинаться с нуля.
Допустимо оставить число a без изменений.
В первой строке входных данных записано натуральное число a (1 ≤ a ≤ 1018). Во второй строке записано натуральное число b (1 ≤ b ≤ 1018). Числа не содержат лидирующих нулей. Гарантируется, что ответ существует.
Выведите наибольшее число, не превосходящее b, полученное перестановкой цифр числа a. Ответ не может начинаться с нуля. Гарантируется, что такое число существует.
Число в выводе должно иметь в точности ту же длину, что и число a. Оно должно являться перестановкой цифр числа a.
 



Далее идут блоки с кодом для телеграм-бота.

In [110]:
!pip3 install keyboa

In [111]:
import keyboa
from keyboa import Keyboa

tasks_ids = [
    {"2-sat": "2-sat"}, {"бинарный поиск": "binary search"}, {"битмаски": "bitmasks"},
    {"перебор": "brute force"}, {"китайская теорема об остатках": "chinese remainder theorem"}, {"комбинаторика": "combinatorics"},
    {"конструктив": "constructive algorithms"}, {"структуры данных": "data structures"}, {"поиск в глубину и т.п.": "dfs ans similar"}, 
    {"разделяй и властвуй": "divide and conquer"}, {"динамика": "dp"}, {"снм": "dsu"},
    {"разбор выражений": "expression parsing"}, {"фурье": "fft"}, {"потоки": "flows"},
    {"игры": "games"}, {"геометрия": "geometry"}, {"паросочетания": "graph matchings"},
    {"графы": "graphs"}, {"жадные алгоритмы": "greedy"}, {"хэши": "hashing"},
    {"реализация": "implementation"}, {"интерактив": "interactive"}, {"математика": "math"},
    {"матрицы": "matrices"}, {"meet-in-the-middle": "meet-in-the-middle"}, {"теория чисел": "number theory"},
    {"теория вероятностей": "probabilities"}, {"расписания": "schedules"}, {"кратчайшие пути": "shortest paths"},
    {"сортировки": "sortings"}, {"строковые суфф. структуры": "string suffix structures"}, {"строки": "strings"},
    {"тернарный поиск": "ternary search"}, {"деревья": "trees"}, {"два указателя": "two pointers"},
    ]

@myBot.callback_query_handler(func=lambda call: True)
def callback_tag(call):
    global tags_
    tags_ = call.data

def sent_task(message):
    str = message.text
    min_ = int (str[0 : str.find('-')])
    max_ = int (str[str.find('-') + 1 : ])
    request = SampleTask.from_csv('myTasks.csv') 
    userTask = request(tags_, min_, max_)
    if userTask.shape[1] == 17:
        myBot.send_message(message.from_user.id, "Задачи с такими параметрами не существует, попробуй еще =)")
        return  
    task = GetTask(userTask)
    myBot.send_message(message.from_user.id, task)
@myBot.message_handler(content_types=['text'])
def get_text_messages(message):
    if message.text == "/help":
        myBot.send_message(message.from_user.id, "Для получения задачи нажми /get_task")
    elif message.text == "/get_task":
        kb_tasks = Keyboa(items=tasks_ids, items_in_row=3).keyboard
        myBot.send_message(message.from_user.id, reply_markup=kb_tasks, text="~ Выбери тему ~")
        myBot.send_message(message.from_user.id, "Напиши диапазон сложности задачи через тире \nНапример: 0-2000")
        myBot.register_next_step_handler(message, sent_task)
    else:
        myBot.send_message(message.from_user.id, "Привет =) \nнапиши /help")

myBot.polling(none_stop=True, interval=0)